In [ ]:
source(paste0(dirname(dirname(dirname(getwd()))),'/map.r'))
source(paste0(HELP_DIR, "shortcuts.r"))
source(paste0(HELP_DIR, "helpers.r"))
source(paste0(HELP_DIR, "fisher.r"))

In [ ]:
library(survminer)
library(rlang)
library(cowplot)
library(patchwork)
library(ggpubr)

# Read in base data

In [ ]:
ready <- 
fread(paste0(SHARE_DIR, "biomarkers_base.csv")) %>% 
 fi(!is.na(purity), !is.na(durableClinicalBenefit)) %>% 
 mu(nrBor = abs(bestOverallResponse-1), non_response = abs(durableClinicalBenefit-1)) %>%
 se(-contains("geneset_mp_"))

In [ ]:
cohorts <- fread(paste0(SHARE_DIR, "fisher_base.csv")) %>% se(sampleId, cohortGo)

In [ ]:
highlights <- 
fread(paste0(SHARE_DIR,"share_highlights_with_fran_update.csv")) %>% 
 fi(!grepl("and_", feature)) %>% 
 se( cohortGo, feature) %>%
 rw() %>% 
 mu( feature_cont = strsplit(feature, "_gt")[[1]][1], 
     feature_cont = strsplit(feature_cont, "_lt")[[1]][1],
     feature_cont = strsplit(feature_cont, "_amp")[[1]][1]) %>% 
 ug() %>% 
 se(-feature) %>% 
 unique()

In [ ]:
#highlights

In [ ]:
features <- highlights %>% pu(feature_cont)

In [ ]:
s1 <- unlist(lapply(features, function(i) strsplit(i, "_gt")[[1]][1]))
s2 <- unlist(lapply(s1, function(i) strsplit(i, "_lt")[[1]][1]))
features_cont <- unlist(lapply(s2, function(i) strsplit(i, "_amp")[[1]][1])) %>% unique()

In [ ]:
go <-
ready %>% 
 se(sampleId, non_response, any_of(features_cont)) %>% 
 lj(cohorts, by = "sampleId") %>% 
 se(-sampleId) %>% 
 ga(feature_cont, val, -cohortGo, -non_response) %>%
 drop_na() %>% 
 ij(highlights, by = c("cohortGo", "feature_cont"))  %>% 
 mu(cohortGo = gsub(" ## ", " / ", cohortGo), 
    non_response = as.factor(ifelse(non_response == 1, "No DCB", "DCB")),
    feature_cont = gsub("Rna", "RNA", str_to_title(gsub("_", " ", gsub("geneset_", "", feature_cont)))), 
    feature_cont = ifelse(feature_cont == "Neo Ct", "RNA Predicted Neoantigens", feature_cont))

- Combine

In [ ]:
base_theme <-
go_theme +
 theme(
  plot.title = element_text(hjust = 0.5), 
  panel.grid.major = element_blank(),  
  panel.grid.minor = element_blank(),
  #axis.title.y = element_blank(), 
  legend.position = "none", 
  axis.text.x = element_text(angle = 0, size = 12, hjust = .5), 
  axis.text.y = element_text(size = 12))

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 4)

In [ ]:
colors <- c("No DCB" = "#e52f28", "DCB" = "#7AABD3")

In [ ]:
share <- 
go %>% 
 fi(grepl("RNA", feature_cont) | feature_cont == "Neo Ct") %>% 
 ggplot( aes(x = as.factor(non_response), y = val, fill = non_response)) + 
 base_theme + 
 scale_fill_manual( values = unlist(colors)) + 
 geom_boxplot(alpha = .6) + 
 facet_wrap(~feature_cont + cohortGo, scales = "free", ncol = 5) + 
 labs(y = "Values", x = "Response Status", title = "Continuous Univariate Features Highlighted")

In [ ]:
ggsave(paste0(SHARE_DIR, "6_continuous_univariate_features.png"), plot = share, width = 16, height = 4)